In [17]:
import time
import requests
from BeautifulSoup import BeautifulSoup
import numpy as np
import pandas as pd
from __future__ import division

In [2]:
def getSessionRequest(email, password):
    session_requests = requests.session()
    login_url = 'https://www.alexa.com/secure/login'
    credentials = {'email': email, 'password': password}
    logged_in = session_requests.post(login_url, data=credentials)
    return session_requests

In [3]:
def getTableFromDomain(domain, session_requests):
    BASE_URL = 'http://www.alexa.com/siteinfo/'
    url = BASE_URL + domain
    response = session_requests.get(url)
    html = response.content
    soup = BeautifulSoup(html)
    table = soup.find('div', attrs={'id': 'demographics-content'})
    return table

In [4]:
def getPercentageFromTags(tag):
    words = tag.split()
    percentage = []
    for word in words:
        if "width" in word:
            percentage.append(word)
    return percentage

In [5]:
def getPercentageFromTable(table):
    tag_gender_age_children = table.findAll("span")[0].prettify()
    tag_education_income = table.findAll("span")[148].prettify()
    tag_browsing_ethnicity = table.findAll("span")[263].prettify()
    gender_age_children_percentage = getPercentageFromTags(tag_gender_age_children)
    education_income_percentage = getPercentageFromTags(tag_education_income)
    browsing_ethnicity_percentage = getPercentageFromTags(tag_browsing_ethnicity)
    return gender_age_children_percentage, education_income_percentage, browsing_ethnicity_percentage

In [6]:
def convertTextIntoInt(percentage):
    final_percentage = []
    for percent in percentage:
        final_percentage.append(percent[(percent.index(":")+1):percent.index("%")])
    final_percentage = map(int, final_percentage)
    return final_percentage

In [7]:
def getRealDemoFromPercentage(gender_age_children_percentage, education_income_percentage, browsing_ethnicity_percentage):
    final_gender_age_children= convertTextIntoInt(gender_age_children_percentage)
    final_education_percentage= convertTextIntoInt(education_income_percentage)
    final_browsing_ethnicity= convertTextIntoInt(browsing_ethnicity_percentage)
    final_percentage = final_gender_age_children + final_education_percentage + final_browsing_ethnicity
    real_demo = [0]*28
    i = 0
    count = 0
    while i < 28:
        real_demo[i] = (final_percentage[count]+final_percentage[count+1])/200
        i = i + 1
        count = count + 2
    return real_demo

In [11]:
def makeDfFromRealDemo(domain_name, real_demo):
    dict = {domain_name:real_demo}
    df=pd.DataFrame.from_dict(dict, orient='index', dtype=None)
    return df

In [13]:
customer1["domain"][0:30]

0       coolmath-games.com
1              scribol.com
2        offbeat.topix.com
3               roblox.com
4              answers.com
5        video.foxnews.com
6        pawsome.topix.com
7           games.aarp.org
8                   y8.com
9              youtube.com
10         silvergames.com
11             foxnews.com
12          boredpanda.com
13         stars.topix.com
14       totaljerkface.com
15      worldlifestyle.com
16    parenthood.topix.com
17                viki.com
18           woozworld.com
19            mirror.co.uk
20                kizi.com
21               women.com
22                 ijr.com
23             realtor.com
24          game3.pogo.com
25            brainjet.com
26          therichest.com
27       thedailybeast.com
28        wunderground.com
29             weather.com
Name: domain, dtype: object

In [18]:
customer1 = pd.read_csv('domain_count.txt', sep='\t',header = None, names=["domain", "count"])
final_df = pd.DataFrame()
session_request = getSessionRequest('alex@kazim.org', 'y0ur0ck!')
count=0
for single_domain in customer1["domain"][0:30]:
    table = getTableFromDomain(single_domain, session_request)
    triple_element = getPercentageFromTable(table)
    e_1 = triple_element[0]
    e_2 = triple_element[1]
    e_3 = triple_element[2]
    real_demo = getRealDemoFromPercentage(e_1, e_2, e_3)
    df = makeDfFromRealDemo(single_domain, real_demo)
    final_df=final_df.append(df)
    count+=1
    time.sleep(1)
    print count

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30


In [19]:
final_df

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
coolmath-games.com,0.260,0.960,0.435,0.410,0.570,0.670,0.530,0.580,0.685,0.365,...,0.315,0.990,0.390,0.060,0.070,0.725,0.965,0.055,0.030,0.050
scribol.com,0.560,0.445,0.230,0.345,0.485,0.805,0.990,0.990,0.470,0.530,...,0.540,0.045,0.710,0.145,0.820,0.105,0.990,0.490,0.300,0.340
offbeat.topix.com,0.310,0.810,0.390,0.450,0.535,0.640,0.625,0.660,0.530,0.470,...,0.385,0.945,0.445,0.145,0.240,0.480,0.990,0.170,0.075,0.120
roblox.com,0.330,0.760,0.490,0.410,0.550,0.620,0.525,0.515,0.675,0.370,...,0.400,0.990,0.370,0.195,0.195,0.620,0.835,0.200,0.155,0.180
answers.com,0.365,0.685,0.445,0.410,0.515,0.620,0.725,0.760,0.490,0.490,...,0.445,0.740,0.455,0.215,0.295,0.475,0.990,0.255,0.125,0.200
video.foxnews.com,0.510,0.490,0.340,0.425,0.510,0.680,0.855,0.855,0.470,0.535,...,0.445,0.490,0.570,0.240,0.400,0.270,0.990,0.405,0.150,0.235
pawsome.topix.com,0.310,0.810,0.390,0.450,0.535,0.640,0.625,0.660,0.530,0.470,...,0.385,0.945,0.445,0.145,0.240,0.480,0.990,0.170,0.075,0.120
games.aarp.org,0.270,0.920,0.350,0.390,0.510,0.705,0.890,0.990,0.575,0.435,...,0.345,0.990,0.415,0.065,0.155,0.595,0.990,0.150,0.030,0.055
y8.com,0.680,0.365,0.605,0.430,0.585,0.405,0.360,0.520,0.790,0.315,...,0.810,0.310,0.345,0.990,0.600,0.470,0.240,0.705,0.515,0.895
youtube.com,0.665,0.375,0.475,0.535,0.490,0.465,0.510,0.455,0.445,0.560,...,0.585,0.180,0.550,0.755,0.415,0.360,0.465,0.770,0.545,0.775


In [20]:
final_df.columns = ["male", "female", "18-24", "25-34", "35-44", "45-54", "55-64", "65+", "Has Children", "No Children", 
"No_college", "Some_college", "Graduate_School", "College", "$0 - $30K", "$30K - $60K", "$60K - $100K", "$100K+", "Home", "School",
"Work", "African", "African American", "Asian", "Caucasian", "Hispanic", "Middle Eastern", "Other"]
final_df["Count"] = customer1["count"][0:30].values



In [21]:
pd.set_option('display.max_columns', 29)
final_df

,male,female,18-24,25-34,35-44,45-54,55-64,65+,Has Children,No Children,No_college,Some_college,Graduate_School,College,$0 - $30K,$30K - $60K,$60K - $100K,$100K+,Home,School,Work,African,African American,Asian,Caucasian,Hispanic,Middle Eastern,Other,Count
coolmath-games.com,0.260,0.960,0.435,0.410,0.570,0.670,0.530,0.580,0.685,0.365,0.810,0.830,0.250,0.420,0.145,0.395,0.905,0.815,0.315,0.990,0.390,0.060,0.070,0.725,0.965,0.055,0.030,0.050,9268406
scribol.com,0.560,0.445,0.230,0.345,0.485,0.805,0.990,0.990,0.470,0.530,0.190,0.525,0.480,0.715,0.285,0.570,0.465,0.815,0.540,0.045,0.710,0.145,0.820,0.105,0.990,0.490,0.300,0.340,7639125
offbeat.topix.com,0.310,0.810,0.390,0.450,0.535,0.640,0.625,0.660,0.530,0.470,0.600,0.725,0.315,0.490,0.205,0.475,0.760,0.730,0.385,0.945,0.445,0.145,0.240,0.480,0.990,0.170,0.075,0.120,6034821
roblox.com,0.330,0.760,0.490,0.410,0.550,0.620,0.525,0.515,0.675,0.370,0.745,0.760,0.295,0.440,0.225,0.445,0.790,0.725,0.400,0.990,0.370,0.195,0.195,0.620,0.835,0.200,0.155,0.180,5815526
answers.com,0.365,0.685,0.445,0.410,0.515,0.620,0.725,0.760,0.490,0.490,0.515,0.660,0.390,0.490,0.295,0.490,0.660,0.655,0.445,0.740,0.455,0.215,0.295,0.475,0.990,0.255,0.125,0.200,5243867
video.foxnews.com,0.510,0.490,0.340,0.425,0.510,0.680,0.855,0.855,0.470,0.535,0.380,0.580,0.415,0.590,0.220,0.510,0.640,0.800,0.445,0.490,0.570,0.240,0.400,0.270,0.990,0.405,0.150,0.235,3822694
pawsome.topix.com,0.310,0.810,0.390,0.450,0.535,0.640,0.625,0.660,0.530,0.470,0.600,0.725,0.315,0.490,0.205,0.475,0.760,0.730,0.385,0.945,0.445,0.145,0.240,0.480,0.990,0.170,0.075,0.120,3792292
games.aarp.org,0.270,0.920,0.350,0.390,0.510,0.705,0.890,0.990,0.575,0.435,0.705,0.800,0.290,0.430,0.150,0.420,0.840,0.830,0.345,0.990,0.415,0.065,0.155,0.595,0.990,0.150,0.030,0.055,3722262
y8.com,0.680,0.365,0.605,0.430,0.585,0.405,0.360,0.520,0.790,0.315,0.390,0.515,0.510,0.545,0.815,0.490,0.320,0.310,0.810,0.310,0.345,0.990,0.600,0.470,0.240,0.705,0.515,0.895,3704292
youtube.com,0.665,0.375,0.475,0.535,0.490,0.465,0.510,0.455,0.445,0.560,0.285,0.390,0.650,0.555,0.645,0.575,0.350,0.395,0.585,0.180,0.550,0.755,0.415,0.360,0.465,0.770,0.545,0.775,3613149
